# Proyecto Final: Predicciones de muertes por COVID-19 

### Introducción

Se utilizará una base de datos llamada **DataSet.csv** que contiene fechas, cantidad de muertes confirmadas y cantidad de contagios confirmados por COVID-19. A este data set lo procesaremos para limpiar los datos y utilizar los óptimos para nuestro entrenamiento que nos servirá para predecir cuantas muertes por día se estiman que habrá.

### Instalación de librerias

Utilizaremos las siguientes librerias para realizar todos nuestros calculos y los analisis.

In [2]:
pip install autots

     |████████████████████████████████| 460 kB 4.2 MB/s 


In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
from autots import AutoTS

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Leer Dataset

Utilizaremos la librería de pandas para leer el archivo CSV, guardarlo en la variable *data* e imprimiremos los primeros cinco datos para ver la forma que tiene la base de datos. 


In [18]:
data = pd.read_csv("DataSet.csv")

print(data.head())

     Date_YMD            Date  Daily Confirmed  Daily Deceased
0  2020-08-18  18/August/2020            16470             700
1  2020-08-19  19/August/2020            15620             694
2  2020-08-20  20/August/2020            14386             699
3  2020-08-21  21/August/2020            15562             672
4  2020-08-22  22/August/2020             7124             563


Checamos que los datos no contengan errores como que sea nulos

In [19]:
data.isnull().sum()

Date_YMD           0
Date               0
Daily Confirmed    0
Daily Deceased     0
dtype: int64

Eliminaremos la columna de *Date* porque no le podremos dar un uso al ya tener la columna *Date_YMD* la cual tiene el formato que deseamos. 
Después graficaremos los datos de casos confirmados por fecha.

In [20]:
data = data.drop("Date", axis=1)

fig = px.bar(data, x='Date_YMD', y='Daily Confirmed',title="Casos confirmados por COVID-19")
fig.show()

De acuerdo con la gráfica se puede notar un gran crecimiento de casos confirmados después de enero de 2022, siendo en enero 10 el día en que más se confirmaron casos de COVID-19. 

Crearemos variables para guardar la cantidad de casos confirmados y de muertes por COVID-19 registrados en nuestro data set y lo graficaremos para poder ver la comparación que existe entre ellos. 

In [16]:
cases = data["Daily Confirmed"].sum()
deceased = data["Daily Deceased"].sum()

labels = ["Confirmed", "Deceased"]
values = [cases, deceased]

fig = px.pie(data, values=values, 
             names=labels, 
             title='Daily Confirmed Cases vs Daily Deaths', hole=0.5)
fig.show()

Podemos notar que la proporción que existe entre las muertes y los casos confirmados es muy diferente, siendo que existen muchísimos más casos confirmados desde que se empezó a llevar registro en septiembre de 2020. 

Calcularemos el índice de mortalidad tomando en cuenta la relación que existe entre la cantidad de muertos y los casos confirmados. De acuerdo con los cálculos siguientes al día mueren de 3 personas contagiadas por COVID-19. 

In [21]:
death_rate = (data["Daily Deceased"].sum() / data["Daily Confirmed"].sum()) * 100
print(death_rate)

2.290727176027623


Graficaremos la cantidad de muertes por fecha

In [22]:
fig = px.bar(data, x='Date_YMD', y='Daily Deceased', title='Muertes confirmadas por COVID-19')
fig.show()

Podemos notar en la grafica anterior que existe un gran decremento de muertes desde septiembre del 2020 a enero del 2022. Notando que existen un aumento muy grande durante SEPTIEMBRE/2020 - MARZO/2021 y JULIO-SEPTIEMBRE/2021 en el que podriamos decir que se le conoce como otra ola de la pandemia.

### Procesamiento

Se utilizará AutoTS para crear el modelo de series de tiempo que se encargará de realizar el pronóstico de alta precisión.  

Se establecerán los siguientes parámetros: 

* *forecast_length*: Es el período para el que queremos predecir la cantidad de defunciones por COVID-19. (Se pondra en 30 ya que queremos obtenerlo para los próximos 30 días.) 

* *frequency* 

* *ensemble*: Le asignaremos *simple* ya que es la forma de conjunto más reconocible y es el promedio de los modelos especificados. 

  

Después de haber creado el modelo, lo ajustaremos a nuestros datos guardados en *data*. También imprimiremos nuestro modelo, esta etapa llevará tiempo ya que se ejecutarán nuestros datos a través de diferentes modelos y verificará el mejor modelo para nuestros datos. 

In [23]:
model = AutoTS(forecast_length=30, 
               frequency='infer', 
               ensemble='simple')
model = model.fit(data, 
                  date_col="Date_YMD", 
                  value_col='Daily Deceased', 
                  id_col=None)

Inferred frequency is: D
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
ETS error TypeError("__init__() got an unexpected keyword argument 'damped_trend'")
Model Number: 9 with model ETS in generation 0 of 10
ETS error TypeError("__init__() got an unexpected keyword argument 'damped_trend'")
Model Number: 10 with model GLM in generation 0 of 10
Template Eval Error: TypeError("ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''") in model 10: GLM
Model Number: 11 with model GLM in generation 0 of 10
Model Number: 12 with model GLS in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model GluonTS in generation 0 of 10
Template Eval Error: ImportError('GluonTS inst

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 43 with model GLS in generation 0 of 10
Model Number: 44 with model SeasonalNaive in generation 0 of 10
Model Number: 45 with model GLM in generation 0 of 10
Template Eval Error: TypeError("ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''") in model 45: GLM
Model Number: 46 with model ETS in generation 0 of 10
ETS error TypeError("__init__() got an unexpected keyword argument 'damped_trend'")


...

117 - FBProphet with avg smape 78.0: 
Model Number: 118 of 131 with model FBProphet for Validation 2
118 - FBProphet with avg smape 49.49: 
Model Number: 119 of 131 with model ZeroesNaive for Validation 2
119 - ZeroesNaive with avg smape 62.34: 
Model Number: 120 of 131 with model DatepartRegression for Validation 2


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done 1000 out of 1000 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


120 - DatepartRegression with avg smape 21.23: 
Model Number: 121 of 131 with model ZeroesNaive for Validation 2
121 - ZeroesNaive with avg smape 66.57: 
Model Number: 122 of 131 with model UnobservedComponents for Validation 2
122 - UnobservedComponents with avg smape 30.2: 
Model Number: 123 of 131 with model DatepartRegression for Validation 2


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done 300 out of 300 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


123 - DatepartRegression with avg smape 25.02: 
Model Number: 124 of 131 with model UnobservedComponents for Validation 2
124 - UnobservedComponents with avg smape 64.95: 
Model Number: 125 of 131 with model UnobservedComponents for Validation 2
125 - UnobservedComponents with avg smape 10.79: 
Model Number: 126 of 131 with model ZeroesNaive for Validation 2
126 - ZeroesNaive with avg smape 13.05: 
Model Number: 127 of 131 with model ZeroesNaive for Validation 2
127 - ZeroesNaive with avg smape 13.05: 
Model Number: 128 of 131 with model ZeroesNaive for Validation 2
128 - ZeroesNaive with avg smape 13.05: 
Model Number: 129 of 131 with model UnobservedComponents for Validation 2
129 - UnobservedComponents with avg smape 14.33: 
Model Number: 130 of 131 with model UnobservedComponents for Validation 2
130 - UnobservedComponents with avg smape 8.33: 
Model Number: 131 of 131 with model ZeroesNaive for Validation 2
131 - ZeroesNaive with avg smape 19.69: 


Imprimiremos el pronóstico del conjunto de datos. Se pronosticarán 30 datos que serán las predicciones de los próximos 30 días a partir de la última fecha del dataset. 

In [24]:
prediction = model.predict()
forecast = prediction.forecast
print(forecast)

            Daily Deceased
2022-01-30          257.25
2022-01-31          352.50
2022-02-01          294.00
2022-02-02          277.50
2022-02-03          266.50
2022-02-04          268.00
2022-02-05          208.75
2022-02-06          210.75
2022-02-07          311.00
2022-02-08          266.00
2022-02-09          234.50
2022-02-10          223.50
2022-02-11          231.00
2022-02-12          175.25
2022-02-13          186.50
2022-02-14          299.50
2022-02-15          221.00
2022-02-16          207.00
2022-02-17          196.00
2022-02-18          198.00
2022-02-19          157.50
2022-02-20          160.50
2022-02-21          288.25
2022-02-22          211.50
2022-02-23          190.00
2022-02-24          188.00
2022-02-25          195.50
2022-02-26          152.75
2022-02-27          161.25
2022-02-28          266.50


### Conclusion


De esta manera es como creamos un modelo utilizando AutoTS para análisis, predicción y pronóstico de series de tiempo. 

Este método es rápido y ahorra tiempo porque no tiene que definir modelos explícitamente ya que hay más de 30 modelos definidos y la lista es cada vez mayor. 

Se llego a crear un modelo de predicción de muertes por COVID-19 en que nos da un aproximado de las cantidades de defunciones que se esperan para cada día desde el 30/ENERO/22 hasta 28/FEBRERO/22. 